<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/basinTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: @ericslevenson
date: Apr 27, 2024
description: Time series of lake extent within watersheds across the Arctic
'''

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project='ee-eric-levenson') # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

# Some common imports
from IPython.display import Image
import folium

Mounted at /content/drive


In [ ]:
ids=[8100069400, 8100069360, 8100353000, 8100033760, 8100033770, 8100033780, 8100033790, 8100353660, 8100353870, 8100033800, 8100033830, 8100354380, 8100354440, 8100033840, 8100033850, 8100033860, 8100033870, 8100033880, 8100033890, 8100073850, 8100073840, 8100033900, 8100033910, 8100355680, 8100033920, 8100033930, 8100355850, 8100033940, 8100033950, 8100033960, 8100033970, 8100033980, 8100033990, 8100034000, 8100034010, 8100076770, 8100076740, 8100076250, 8100076260, 8100356590, 8100356360, 8100356320, 8100034020, 8100077940, 8100077980, 8100034040, 8100034050, 8100077900, 8100077910, 8100078020, 8100078040, 8100078060, 8100078090, 8100357980, 8100034060, 8100034070, 8100034080, 8100034090, 8100078520, 8100078550, 8100078700, 8100078750, 8100358120, 8100079120, 8100079160, 8100358220, 8100358210, 8100357830, 8100358340, 8100358250, 8100358500, 8100079980, 8100079260, 8100079210, 8100080060, 8100080090, 8100079270, 8100079190, 8100079170, 8100358290, 8100358370, 8100358050, 8100078810, 8100078770, 8100358620, 8100357890, 8100358740, 8100078480, 8100078460, 8100078450, 8100078430, 8100078360, 8100357560, 8100034100, 8100034110, 8100034120, 8100034130, 8100034140, 8100034150, 8100356980, 8100357080, 8100034160, 8100034170, 8100075500, 8100075510, 8100075950, 8100075980, 8100075960, 8100075930, 8100076960, 8100077040, 8100076150, 8100076170, 8100077400, 8100077420, 8100356900, 8100077640, 8100077690, 8100076360, 8100076310, 8100357570, 8100357240, 8100076000, 8100075990, 8100075880, 8100075790, 8100075800, 8100077050, 8100077100, 8100356540, 8100076780, 8100076750, 8100076590, 8100076560, 8100077060, 8100077110, 8100034180, 8100034190, 8100075290, 8100075320, 8100034200, 8100034210, 8100074260, 8100074290, 8100355790, 8100355820, 8100074320, 8100074360, 8100074230, 8100074220, 8100074110, 8100074090, 8100073820, 8100073830, 8100355610, 8100073590, 8100073560, 8100074330, 8100074370, 8100073300, 8100355390, 8100074800, 8100074850, 8100034230, 8100355510, 8100355370, 8100355200, 8100355290, 8100034240, 8100034250, 8100034260, 8100034270, 8100354850, 8100034280, 8100034290, 8100034300, 8100034310, 8100034320, 8100034330, 8100354880, 8100034340, 8100034350, 8100073160, 8100073130, 8100354860, 8100355060, 8100354670, 8100354460, 8100072310, 8100072300, 8100072710, 8100072810, 8100072800, 8100034360, 8100034370, 8100072530, 8100072630, 8100034380, 8100034390, 8100071380, 8100071410, 8100034400, 8100034410, 8100070930, 8100070910, 8100070770, 8100070760, 8100070510, 8100070490, 8100353610, 8100353510, 8100070520, 8100070530, 8100070710, 8100353410, 8100353320, 8100034420, 8100034430, 8100353390, 8100353370, 8100034450, 8100034460, 8100034470, 8100034480, 8100034490, 8100034500, 8100034510, 8100034520, 8100034530, 8100353260, 8100034540, 8100034550, 8100034560, 8100034570, 8100034580, 8100034590, 8100034600, 8100034610, 8100068860, 8100068840, 8100034620, 8100034630, 8100352420, 8100034640, 8100034650, 8100352290, 8100034660, 8100034670, 8100352220, 8100352310, 8100034680, 8100034700, 8100034720, 8100034730, 8100034740, 8100034750, 8100351700, 8100351850, 8100034760, 8100034770, 8100034780, 8100034790, 8100034800, 8100034810, 8100351750, 8100352000, 8100352160, 8100034820, 8100034830, 8100034840, 8100034850, 8100352190, 8100034860, 8100034870, 8100034890, 8100069200, 8100069260, 8100069170, 8100069190, 8100069300, 8100069310, 8100068940, 8100069110, 8100352980, 8100068900, 8100353090, 8100034900, 8100034910, 8100034920, 8100034930, 8100070080, 8100034940, 8100034950, 8100034960, 8100034970, 8100071210, 8100071190, 8100071170, 8100071140, 8100353530, 8100353430, 8100034980, 8100034990, 8100035000, 8100035010, 8100072700, 8100072670, 8100354540, 8100354370, 8100072980, 8100072990, 8100073410, 8100073460, 8100071860, 8100071890, 8100355300, 8100355350, 8100072350, 8100072370, 8100071370, 8100071360, 8100073960, 8100073990, 8100355450, 8100353840, 8100353860, 8100355540, 8100074190, 8100074180, 8100074380, 8100074390, 8100353830, 8100355160, 8100355990, 8100354950, 8100035020, 8100035030, 8100073940, 8100073970, 8100074000, 8100074010, 8100355690, 8100355660, 8100356050, 8100356020, 8100035040, 8100035050, 8100035070, 8100355000, 8100355250, 8100355330, 8100355710, 8100355980, 8100355770, 8100035080, 8100035090, 8100354740, 8100035100, 8100035110, 8100035120, 8100035130, 8100035140, 8100035150, 8100073240, 8100073230, 8100035160, 8100035170, 8100035180, 8100035190, 8100355360, 8100035200, 8100035210, 8100035220, 8100074070, 8100074100, 8100355630, 8100355780, 8100074440, 8100074450, 8100073800, 8100073760, 8100035260, 8100035270, 8100354960, 8100035290, 8100035300, 8100035310, 8100035320, 8100072650, 8100072570, 8100035340, 8100035350, 8100035360, 8100035370, 8100070570, 8100070620, 8100353560, 8100353540, 8100353930, 8100353590, 8100353600, 8100354020, 8100353750, 8100035380, 8100035390, 8100035400, 8100035410, 8100353400, 8100035440, 8100035450, 8100035470, 8100035480, 8100035490, 8100035500, 8100035510, 8100035520, 8100035530, 8100035540, 8100035550, 8100074150, 8100035560, 8100035580, 8100035590, 8100074310, 8100074560, 8100074600, 8100074770, 8100074810, 8100035600, 8100035610, 8100035630, 8100354910, 8100035650, 8100035660, 8100035670, 8100035680, 8100035700, 8100035710, 8100035720, 8100075210, 8100075230, 8100356450, 8100356640, 8100356870, 8100035730, 8100035740, 8100035750, 8100035760, 8100035770, 8100035780, 8100035800, 8100076800, 8100076820, 8100035810, 8100035820, 8100035830, 8100035840, 8100035850, 8100035860, 8100035870, 8100356770, 8100356950, 8100357210, 8100035890, 8100357280, 8100035900, 8100035910, 8100035920, 8100035930, 8100035940, 8100035950, 8100035960, 8100035970, 8100077790, 8100077780, 8100035980, 8100036000, 8100036010, 8100036020, 8100077970, 8100036030, 8100076670, 8100076710, 8100077600, 8100077590, 8100036060, 8100036070, 8100036100, 8100036110, 8100036120, 8100036130, 8100356460, 8100036160, 8100036170, 8100036180, 8100036190, 8100356880, 8100356800, 8100036200, 8100036210, 8100036220, 8100036230, 8100036250, 8100036260, 8100036280, 8100036290, 8100036300, 8100080520, 8100080530, 8100358790, 8100081030, 8100080950, 8100081150, 8100081140, 8100081680, 8100081750, 8100081960, 8100081970, 8100081990, 8100082060, 8100082050, 8100082310, 8100082320, 8100082800, 8100082790, 8100359150, 8100081020, 8100080940, 8100080610, 8100080600, 8100081290, 8100081280, 8100080100, 8100080070, 8100358230, 8100036310, 8100036340, 8100358130, 8100036350, 8100036360, 8100036370, 8100036380, 8100036390, 8100036400, 8100036410, 8100036420, 8100082710, 8100082760, 8100083450, 8100083480, 8100084240, 8100084290, 8100084400, 8100084390, 8100360660, 8100360850, 8100360270, 8100085430, 8100085380, 8100036430, 8100036440, 8100360670, 8100086030, 8100086040, 8100361640, 8100361460, 8100361370, 8100361250, 8100036450, 8100036460, 8100084830, 8100084800, 8100036470, 8100036480, 8100036500, 8100036520, 8100036530, 8100036540, 8100036550, 8100036560, 8100358550, 8100036570, 8100036580, 8100358800, 8100358890, 8100359050, 8100081480, 8100081530, 8100359190, 8100359280, 8100359700, 8100083640, 8100360600, 8100036590, 8100036600, 8100359080, 8100036610, 8100036620, 8100036630, 8100036640, 8100036650, 8100036700, 8100036710, 8100036720, 8100087030, 8100087090, 8100087520, 8100087570, 8100086900, 8100086950, 8100087670, 8100087690, 8100087770, 8100087870, 8100088340, 8100088380, 8100088790, 8100088780, 8100089670, 8100089660, 8100089750, 8100089860, 8100089410, 8100089370, 8100362120, 8100036730, 8100036770, 8100036790, 8100036800, 8100036810, 8100036830, 8100036840, 8100036850, 8100036870, 8100036880, 8100036890, 8100036900, 8100036910, 8100036920, 8100360780, 8100036930, 8100036940, 8100085850, 8100085840, 8100036950, 8100036960, 8100361710, 8100036970, 8100036980, 8100036990, 8100037000, 8100037010, 8100037020, 8100363090, 8100363040, 8100090490, 8100090610, 8100090640, 8100363520, 8100363780, 8100092030, 8100092130, 8100364050, 8100364500, 8100037030, 8100037050, 8100037060, 8100037070, 8100037080, 8100037090, 8100037100, 8100037110, 8100363980, 8100037120, 8100037130, 8100037140, 8100037150, 8100037170, 8100037190, 8100037200, 8100037210, 8100364570, 8100364620, 8100093520, 8100093660, 8100364360, 8100364250, 8100364060, 8100363950, 8100037220, 8100037230, 8100037240, 8100037250, 8100037260, 8100097350, 8100097290, 8100037280, 8100037290, 8100037300, 8100037310, 8100037320, 8100037330, 8100366720, 8100037340, 8100037350, 8100100520, 8100100590, 8100101800, 8100101860, 8100103360, 8100103420, 8100103260, 8100103210, 8100368830, 8100369560, 8100037370, 8100367550, 8100037380, 8100037390, 8100103560, 8100103620, 8100103980, 8100103920, 8100037400, 8100037410, 8100103870, 8100103860, 8100104940, 8100104970, 8100105910, 8100105830, 8100370800, 8100371550, 8100371830, 8100037420, 8100037430, 8100037440, 8100037450, 8100037470, 8100371020, 8100037480, 8100037490, 8100371700, 8100371370, 8100109460, 8100109450, 8100372040, 8100372330, 8100372540, 8100371950, 8100372420, 8100037500, 8100037510, 8100037520, 8100037530, 8100037540, 8100037550, 8100037560, 8100373960, 8100117180, 8100116960, 8100117720, 8100117730, 8100118150, 8100118350, 8100118200, 8100037580, 8100037600, 8100037610, 8100037620, 8100037630, 8100037640, 8100121280, 8100037650, 8100037660, 8100037670, 8100037680, 8100120290, 8100120280, 8100037690, 8100037710, 8100037720, 8100037750, 8100037760, 8100037800, 8100037810, 8100151300, 8100039190, 8100171910, 8100142380, 8100141890, 8100141770, 8100142230, 8100142120, 8100142130, 8100142020, 8100141530, 8100141410, 8100142810, 8100142900, 8100385620, 8100384730, 8100385790, 8100140260, 8100140270, 8100385910, 8100144270, 8100384250, 8100144340, 8100144480, 8100144490, 8100384080, 8100386390, 8100145650, 8100383780, 8100145710, 8100386850, 8100383530, 8100137390, 8100146880, 8100137400, 8100146950, 8100383080, 8100147500, 8100147590, 8100147720, 8100147730, 8100147990, 8100148000, 8100382820, 8100148260, 8100148270, 8100148330, 8100148390, 8100148380, 8100148450, 8100135230, 8100135240, 8100388780, 8100133240, 8100133130, 8100132610, 8100132460, 8100140400, 8100140280, 8100153280, 8100153380, 8100388370, 8100384170, 8100130440, 8100130450, 8100149160, 8100149260, 8100149350, 8100149450, 8100155400, 8100155390, 8100150100, 8100150210, 8100150440, 8100150450, 8100383350, 8100128440, 8100128430, 8100154610, 8100154500, 8100380130, 8100389380, 8100136110, 8100136020, 8100127160, 8100127080, 8100158270, 8100158320, 8100379960, 8100138300, 8100138310, 8100135540, 8100135530, 8100146070, 8100146150, 8100378450, 8100159630, 8100159620, 8100390870, 8100390210, 8100159770, 8100159780, 8100160140, 8100160280, 8100149730, 8100149880, 8100385720, 8100160880, 8100160990, 8100387760, 8100133640, 8100161670, 8100133550, 8100161660, 8100385410, 8100136720, 8100136630, 8100391760, 8100124850, 8100124860, 8100136730, 8100136810, 8100124620, 8100124490, 8100157760, 8100157770, 8100382440, 8100380600, 8100161400, 8100161470, 8100123610, 8100123710, 8100394490, 8100393440, 8100388650, 8100393910, 8100123300, 8100123240, 8100164960, 8100165020, 8100381820, 8100392720, 8100122080, 8100122070, 8100165230, 8100165220, 8100380480, 8100137030, 8100137040, 8100121370, 8100121360, 8100381170, 8100166320, 8100166310, 8100121380, 8100121390, 8100138010, 8100138090, 8100393020, 8100136030, 8100136040, 8100165120, 8100165040, 8100152840, 8100152910, 8100380430, 8100167740, 8100167750, 8100395710, 8100165210, 8100165310, 8100396160, 8100138980, 8100139100, 8100125700, 8100125580, 8100395280, 8100120400, 8100120270, 8100119040, 8100118950, 8100379690, 8100163770, 8100163840, 8100140170, 8100140290, 8100168990, 8100169060, 8100396940, 8100381370, 8100119510, 8100119440, 8100118130, 8100118140, 8100378670, 8100119170, 8100119050, 8100132000, 8100132010, 8100117800, 8100117710, 8100396530, 8100126500, 8100126510, 8100396760, 8100156990, 8100157060, 8100123560, 8100123620, 8100117280, 8100117290, 8100126370, 8100157890]

In [ ]:
## ***INPUTS***

## Regularly Adjusted Inputs ##

# Tile of Interest ID

start = '2016-03-01'
finish = '2023-10-01'

## Periodically Adjusted Inputs
# Lake Shapefiles
lakes = ee.FeatureCollection('projects/ee-eric-levenson/assets/PLD/PLD_60n0e_buff')
lakes = lakes.merge(ee.FeatureCollection('projects/ee-eric-levenson/assets/PLD/PLD_60n110w_buff'))
lakes = lakes.merge(ee.FeatureCollection('projects/ee-eric-levenson/assets/PLD/PLD_60n180w_buff'))
lakes = lakes.merge(ee.FeatureCollection('projects/ee-eric-levenson/assets/PLD/PLD_60n60w_buff'))
lakes = lakes.merge(ee.FeatureCollection('projects/ee-eric-levenson/assets/PLD/PLD_60n90e_buff'))

# Export settings
directory = 'timeSeries' #Export Folder
description_ending = f'_timeSeries' # Export Name after tile ID

#Basins
basin_l10 = ee.FeatureCollection("projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev10")

# exports
exportSelectors = ['id', 'date', 'waterArea', 'coverage']

## Rarely Adjusted Inputs ##
# Image scale
pixScale = 10
# Cloud probability threshold
CLD_PRB_THRESH = 50
## ***EARTH ENGINE-IFY***
startDoy = ee.Date(start).getRelative('day', 'year')
endDoy = ee.Date(finish).getRelative('day', 'year')
eestart = ee.Date(start)
eefinish = ee.Date(finish)

## Functions

### Image Pre-Processing Methods

In [ ]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    clouds = cld_prb.gte(CLD_PRB_THRESH).rename('cloud_mask')
    clear = cld_prb.lt(CLD_PRB_THRESH).rename('clear_mask')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, clouds, clear]))

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

def clip2lakes(image):
  '''Clips an image based on the lake boundaries'''
  return image.clip(lakes)

# Get percentile cover
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actPixels = ee.Number(image.updateMask(image.select('clear_mask')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100, # keep same as totPixels
      geometry = image.geometry(),
      maxPixels=1e12,
      ).values().get(0))
  # calculate the perc of cover OF CLEAR PIXELS
  percCover = actPixels.divide(totPixels).multiply(100).round()
  # number as output
  return image.set('percCover', percCover,'actPixels',actPixels)

# Mosaic images by date, orbit, - basically combines images together that were taken on the same day
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)

### Ice Classification Methods

In [ ]:
def ice_classify(image):
    clear_mask = image.select('clear_mask')
    ice = image.select('B4').gte(1000).rename('ice')  # Addy's threshold
    ice = ice.updateMask(clear_mask)  # Apply clear mask to ice classification
    #all = image.select('B4').gte(1).rename('all')
    return image.addBands([ice])

### Water Classification Methods

In [ ]:
###########################################################################
## ***WATER CLASSIFICATION METHODS***

# Define NDWI image
def ndwi(image):
  '''Adds an NDWI band to the input image'''
  return image.normalizedDifference(['B3', 'B8']).rename('NDWI').multiply(1000)

# Basic ndwi classification
def ndwi_classify(image):
  '''Creates a binary image based on an NDWI threshold of 0'''
  ndwimask = image.select('NDWI')
  water = ndwimask.gte(0)
  land = ndwimask.lt(0)
  return(water)

# OTSU thresholding from histogram
def otsu(histogram):
  '''Returns the NDWI threshold for binary water classification'''
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  indices = ee.List.sequence(1, size)
  def func_xxx(i):
    '''Compute between sum of squares, where each mean partitions the data.'''
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts) \
        .reduce(ee.Reducer.sum(), [0]).get([0]) \
        .divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(
           bCount.multiply(bMean.subtract(mean).pow(2)))
  bss = indices.map(func_xxx)
  # Return the mean value corresponding to the maximum BSS.
  return means.sort(bss).get([-1])

# OTSU thresholding for an image
def otsu_thresh(water_image):
  '''Calculate NDWI and create histogram. Return the OTSU threshold.'''
  NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clear_mask'))
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  return otsu(histogram.get('NDWI_histogram'))

# Classify an image using OTSU threshold.
def otsu_classify(water_image):
  '''(1) Calculate NDWI and create histogram. (2) Calculate NDWI threshold for
  binary classification using OTSU method. (3) Classify image and add layer to input image.
  '''
  NDWI = ndwi(water_image).select('NDWI')
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  threshold = otsu(histogram.get('NDWI_histogram'))
  otsu_classed = NDWI.gt(ee.Number(threshold)).And(water_image.select('B8').lt(2000)).rename('otsu_classed')
  return water_image.addBands([otsu_classed])

def adaptive_thresholding(water_image):
  '''Takes an image clipped to lakes and returns the water mask'''
  NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clear_mask')) # get NDWI **TURNED OFF CLOUD MASK, because i do it in the mains cript
  threshold = ee.Number(otsu_thresh(water_image))
  threshold = threshold.divide(10).round().multiply(10)
  # get fixed histogram
  histo = NDWI.reduceRegion(
      geometry = roi,
      reducer = ee.Reducer.fixedHistogram(-1000, 1000, 200),
      scale = pixScale, # This was 30, keep at 10!?!?
      maxPixels = 1e12
  )
  hist = ee.Array(histo.get('NDWI'))
  counts = hist.cut([-1,1])
  buckets = hist.cut([-1,0])
  #find split points from otsu threshold
  threshold = ee.Array([threshold]).toList()
  buckets_list = buckets.toList()
  split = buckets_list.indexOf(threshold)
  # split into land and water slices
  land_slice = counts.slice(0,0,split)
  water_slice = counts.slice(0,split.add(1),-1)
  # find max of land and water slices
  land_max = land_slice.reduce(ee.Reducer.max(),[0])
  water_max = water_slice.reduce(ee.Reducer.max(),[0])
  land_max = land_max.toList().get(0)
  water_max = water_max.toList().get(0)
  land_max = ee.List(land_max).getNumber(0)
  water_max = ee.List(water_max).getNumber(0)
  #find difference between land, water and otsu val
  counts_list = counts.toList()
  otsu_val = ee.Number(counts_list.get(split))
  otsu_val = ee.List(otsu_val).getNumber(0)
  land_prom = ee.Number(land_max).subtract(otsu_val)
  water_prom = ee.Number(water_max).subtract(otsu_val)
  #find land and water buckets corresponding to 0.9 times the prominence
  land_thresh = ee.Number(land_max).subtract((land_prom).multiply(ee.Number(0.9)))
  water_thresh = ee.Number(water_max).subtract((water_prom).multiply(ee.Number(0.9)))
  land_max_ind = land_slice.argmax().get(0)
  water_max_ind = water_slice.argmax().get(0)
  li = ee.Number(land_max_ind).subtract(1)
  li = li.max(ee.Number(1))
  wi = ee.Number(water_max_ind).add(1)
  wi = wi.min(ee.Number(199))
  land_slice2 = land_slice.slice(0,li,-1).subtract(land_thresh)
  water_slice2 = water_slice.slice(0,0,wi).subtract(water_thresh)
  land_slice2  = land_slice2.abs().multiply(-1)
  water_slice2 = water_slice2.abs().multiply(-1)
  land_index = ee.Number(land_slice2.argmax().get(0)).add(land_max_ind)
  water_index = ee.Number(water_slice2.argmax().get(0)).add(split)
  land_level = ee.Number(buckets_list.get(land_index))
  water_level = ee.Number(buckets_list.get(water_index))
  land_level = ee.Number(ee.List(land_level).get(0)).add(5)
  water_level = ee.Number(ee.List(water_level).get(0)).add(5)
  #calculate water fraction and classify
  water_fraction = (NDWI.subtract(land_level)).divide(water_level.subtract(land_level)).multiply(100).rename('water_fraction')
  #water_fraction = conditional(water_fraction) #sets values less than 0 to 0 and greater than 100 to 100
  water_75 = water_fraction.gte(75).rename('water_75'); #note, this is a non-binary classification, so we use 75% water as "water"
  all_mask = water_image.select('B2').gt(5).rename('all_mask')
  cloud_mask_ed = water_image.select('cloud_mask').rename('cloud_mask_ed')
  return water_image.addBands([water_fraction,water_75,NDWI,cloud_mask_ed])

def binaryImage(image):
  '''takes a multiband image and returns just the binary water_75 band'''
  img = image.select('water_75')
  return img
def waterImage(image):
  '''takes a multiband image and returns just the water fraction band'''
  img = image.select('water_fraction')
  return img

### Property Extraction Methods

In [ ]:

def pixelArea(image):
  areaIm = image.pixelArea()
  lakeIm = image.addBands([areaIm])
  return lakeIm

def sumWater(image):
  '''sums the water pixels within the watershed image and adds the result to the feature'''
  waterAreaIm = image.updateMask(image.select('water_75')) # mask area image based on water
  # calculate the total area
  watersum = waterAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = image.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return watersum

def sumIce(image):
  '''sums the ice pixels within the watershed image and adds the result to the feature'''
  iceAreaIm = image.updateMask(image.select('ice')) # mask area image based on ice
  # calculate the total area
  icesum = iceAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = image.geometry(),
      scale = 100,
      maxPixels=1e9
  ).get('area')
  return icesum

def getClearArea(image):
  '''sums the clear pixels within the watershed image and adds the result to the feature'''
  clearAreaIm = image.updateMask(image.select('clear_mask')) # mask area image based on clearness
  clearArea = clearAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = image.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return clearArea

def getCloudArea(image):
  '''sums the cloud pixels within the watershed image and adds the result to the feature'''
  cloudAreaIm = image.updateMask(image.select('cloud_mask')) # mask area image based on clouds
  cloudArea = cloudAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = image.geometry(),
      scale = 10, ## TODO: I updated this scale from 10 to 100, does that change results?
      maxPixels=1e9
  ).get('area')
  return cloudArea

def dayProps(image):
  '''Input: classified water Image --> Output: Feature with properties such as date, water area, clear area, etc.
  Map this function to the image collection of classified lake area images. This function calls
  methods to calculate the water area, clear area, and cloud area. '''
  basin = id #TODO: FIXME
  date = image.date().format('yyyy-MM-dd')
  water = sumWater(image)
  #ice = sumIce(image)
  cover = image.get('percCover')
  #iceCover = image.get('percIceCover')
  #clear = getClearArea(image)
  #cloud = getCloudArea(image)
  return ee.Feature(None, {'id': basin, 'date': date, 'waterArea': water,  'coverage':cover})

def export_lakes(collection, description, fileNamePrefix, fileFormat, folder, selectors):
  '''Export a feature collection of lake properties to google drive for a given day.'''
  task = ee.batch.Export.table.toDrive(**{
    'collection': collection,
    'description': description,
    'fileNamePrefix': fileNamePrefix,
    'fileFormat': fileFormat,
    'folder': folder,
    'selectors': selectors
  })
  task.start()

## Main

In [ ]:
## ***MAIN***
for i, id in enumerate(ids):
  #(1) define roi
  basin = ee.Feature(basin_l10.filter(ee.Filter.eq('HYBAS_ID', id)).first())
  roi = ee.Geometry.Polygon(basin.geometry().getInfo()['coordinates'][0]) # define roi as geometry variable

  # (2) image pre-process:
  # Get images and filter images by cloudiness, roi, time period, and month range
  images = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',60)) # Get Images
  # Get cloud probability image collection
  s2Cloudless = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year'))
  # Merge surface reflectance and cloud probability collections
  images = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
          'primary': images,
          'secondary': s2Cloudless,
          'condition': ee.Filter.equals(**{
              'leftField': 'system:index',
              'rightField': 'system:index'
          })
      }))
  images = images.map(add_cloud_bands)
  # Mosaic images and add cloud/clear masks
  images_all = mosaicBy(images)
  # Clip mosaics to roi
  images_all = images_all.map(clip_image)
  # Clip remaining mosaics to buffered lake shapefile
  lakeimages = images_all.map(clip2lakes) # Clip images to buffered lake mask

  # Get percent cover for each mosaic
  image_mask = lakeimages.select('B2').mean().gte(0) #
  # Calculate total number of pixels
  totPixels = ee.Number(image_mask.reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      geometry = roi,
      maxPixels = 1e12
      ).values().get(0))
  lakeimages = lakeimages.map(getCover) # add percentage cover as an image property
  # Filter by percent cover
  lakeimages = lakeimages.filterMetadata('percCover','greater_than',70) # remove images covering less than 70% of the ROI)
  # (3) Classify water
  lakeimages = lakeimages.map(adaptive_thresholding)
  lakeimages = lakeimages.map(ice_classify)
  # Convert to area images
  lakeimages = lakeimages.map(pixelArea) # get a pixel area image
  #(4) Calculate water and ice area, convert to table format
  days = lakeimages.map(dayProps)

  #(5) Export
  export_lakes(days, str(id), str(id), 'csv', directory, exportSelectors)


## Storage

In [ ]:
id=8100140540
basin = ee.Feature(basin_l10.filter(ee.Filter.eq('HYBAS_ID', id)).first())
roi = ee.Geometry.Polygon(basin.geometry().getInfo()['coordinates'][0]) # define roi as geometry variable

  # (2) image pre-process:
  # Get images and filter images by cloudiness, roi, time period, and month range
imagesv2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
s2Cloudless = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year'))
ims = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': imagesv2,
        'secondary': s2Cloudless,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

ims = ims.map(add_cloud_bands)
images_allv2 = mosaicBy(ims)
images_allv2 = images_allv2.map(clip_image)
image_maskv2 = images_allv2.select('B2').mean().clip(roi).gte(0) #First, calculate total number of pixels
totPixels = ee.Number(image_maskv2.reduceRegion(
    reducer = ee.Reducer.count(),
    scale = 100,
    geometry = roi,
    maxPixels = 1e12
    ).values().get(0))
images_allv2 = images_allv2.map(getCover)
#images_allv2 = images_allv2.map(applyMask)

In [ ]:
lakeimages = images_allv2.map(clip2lakes) # Clip images to buffered lake mask

  # (3) Classify water
lakeimages = lakeimages.map(adaptive_thresholding)
  # Convert to area images